##### Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf

# Visualization(for Animation) Libraries
import imageio
import matplotlib.pyplot as plt

from datetime import datetime as dt

2023-08-11 09:48:54.358045: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


##### Preparation of Simple Data for Modeling

In [2]:
# train_y = train_x * 3 + 2
train_x = np.array([[5], [-10], [9], [0], [4], [2]])
train_y = np.array([17, -28, 29, 2, 14, 8]) 

##### Model Definition with Deep Learning

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)

2023-08-11 09:48:58.429493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22787 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:5e:00.0, compute capability: 7.5
2023-08-11 09:48:58.430357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22303 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:af:00.0, compute capability: 7.5


##### Model Training

In [4]:
EPOCHS = 300

In [5]:
weights_history = []

_training_start = dt.now()
for _ in range(EPOCHS+1):
    model.fit(train_x, train_y, epochs=1, verbose=0)
    _weights = model.get_weights()
    weights_history.append(_weights) 

print(f'Total Processing Time: {dt.now() - _training_start}')

2023-08-11 09:49:00.508922: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x2c4d2c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-11 09:49:00.508995: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2023-08-11 09:49:00.509020: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (1): NVIDIA TITAN RTX, Compute Capability 7.5
2023-08-11 09:49:00.523493: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-11 09:49:00.753364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-11 09:49:00.826703: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-11 09:49:00.937363: I ./tensorflow/compiler/jit/device_compiler

Total Processing time: 0:00:22.978802


##### Generating Images for Each Epoch

* Make Temp Path

In [9]:
TEMP_DIR_NAME = '_temp_images'
TEMP_PATH = os.path.join(os.getcwd(), TEMP_DIR_NAME)

if not os.path.exists(TEMP_PATH):
    os.makedirs(TEMP_PATH, exist_ok=True)

* Creating Images

In [10]:
_generation_images_start = dt.now()

# Performed 300 iterations
for idx, weights in enumerate(weights_history):     
    model.set_weights(weights)
    _predictions = model.predict(train_x, verbose=0)
    
    plt.figure(figsize=(10, 10))
    plt.title(f'Epoch: {idx}')
    plt.scatter(train_x, train_y, label='Actual')
    plt.scatter(train_x, _predictions, label='Predicted')
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.legend(loc='upper left')
    
    _temp_image_filename = f'temp_{idx:04d}.png'
    plt.savefig(os.path.join(TEMP_PATH, _temp_image_filename))
    
    # None Display Images
    plt.close()
    
print(f'Total Processing Time: {dt.now() - _generation_images_start}')

Total Processing time: 0:01:14.901483


* Creating Animation

In [12]:
ANIMATION_FILENAME = 'training_animation.gif'

_animations = []
frames = [img for img in os.listdir(TEMP_PATH) if img.endswith('.png')]
frames.sort()

_creating_start = dt.now()
for frame in frames:
    _frame_path = os.path.join(TEMP_PATH, frame)
    _mem_frame = imageio.imread(_frame_path)
    _animations.append(_mem_frame)
    os.remove(_frame_path)
    
imageio.mimsave(ANIMATION_FILENAME, _animations, format='GIF', duration=0.5)

print(f'Total Processing Time: {dt.now() - _creating_start}')

try:
    os.rmdir(TEMP_PATH)
except OSError as e:
    print(f'Error deleting directory: {e}')

/tmp/ipykernel_37509/1996359379.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  _mem_frame = imageio.imread(_frame_path)


Total Processing Time: 0:00:13.496046
